# check oof df

In [2]:
import os

import numpy as np
import pandas as pd
from pandarallel import pandarallel
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

In [4]:
output_dir = os.path.join("/kaggle", "working", "_oof")
# exp_name = "baseline000"
# exp_name = "baseline000_1697037446.0796616"
# exp_name = "baseline_alldata_allfold_000"
exp_name = "exp005_lossbugfixed_lr"
# oof_df_path = os.path.join(output_dir, exp_name,"oof_df.parquet")
# oof_df = pd.read_parquet(oof_df_path)
# for i in range(5):
#     print("fold", i)
#     oof_df = pd.concat(
#         [pd.read_parquet(os.path.join(output_dir, exp_name, f"oof_df_fold{i}.parquet"))]
#         ,axis=0
#     )

In [5]:
# postprocess_fn
# series_idでgroupbyして、class_predに対して対象の列のデータから前のN個の列までのデータの平均をとる
import torch
import torch.nn as nn

# 1step 0.5secで30minなら60*30=1800step
def postprocess_fn(df, N=1800, maxpool_kernel_size=101, maxpool_stride=1):
    df = df.copy()
    df["class_pred_beforemean"] = df.groupby("series_id")["class_pred"].apply(lambda x: x.rolling(N, min_periods=1).mean())
    df["class_pred_aftermean"] = df.groupby("series_id")["class_pred"].apply(lambda x: x[::-1].rolling(N, min_periods=1).mean()[::-1])
    df["event_pred"] = df["class_pred_beforemean"] - df["class_pred_aftermean"]

    # 入力サイズと出力サイズが一致するようにpaddingを調整
    maxpool_padding = int((maxpool_kernel_size - maxpool_stride) / 2)
    # maxpoolしてピーク検出
    max_pooling = nn.MaxPool1d(maxpool_kernel_size, stride=maxpool_stride, padding=maxpool_padding)
    event_pred = df["event_pred"].values
    event_pred = torch.tensor(event_pred).unsqueeze(0)
    pooled_event_pred = max_pooling(np.abs(event_pred)).squeeze(0).numpy()
    event_pred = event_pred.squeeze(0).numpy()
    # peakのところだけ残すmaskを作成
    peak_event_pred_mask = np.where(pooled_event_pred == np.abs(event_pred), 1, 0)
    peak_event_pred = event_pred * peak_event_pred_mask
    df["event_pred"] = peak_event_pred
    df = df.drop(["class_pred_beforemean", "class_pred_aftermean"], axis=1)
    return df


def make_submission_df(df, threshold=0.0):
    df = df[["series_id", "step", "event_pred"]].copy()
    # thresholdより大きいときは1,-thresholdより小さいときは-1,それ以外は0
    df["event"] = df["event_pred"].apply(lambda x: 1 if x > threshold else -1 if x < -threshold else 0)
    df = df[df["event"] != 0].copy()
    df["event"] = df["event"].replace({1: "wakeup", -1: "onset"})
    df["score"] = df["event_pred"].apply(lambda x: np.clip(np.abs(x), 0.0, 1.0))
    return df


In [7]:
train_event_df = pd.read_csv("/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv")
# train_event_df = train_event_df[train_event_df["series_id"].isin(oof_df["series_id"].unique())].copy()
# train_event_df = train_event_df.reset_index(drop=True)
display(train_event_df.head())

,series_id,night,event,step,timestamp
0,038441c925bb,1,onset,4992.0,2018-08-14T22:26:00-0400
1,038441c925bb,1,wakeup,10932.0,2018-08-15T06:41:00-0400
2,038441c925bb,2,onset,20244.0,2018-08-15T19:37:00-0400
3,038441c925bb,2,wakeup,27492.0,2018-08-16T05:41:00-0400
4,038441c925bb,3,onset,39996.0,2018-08-16T23:03:00-0400


In [8]:
import sys
sys.path.append("/kaggle/src/dss_utils")

# series_id_column_name = "series_id"
# time_column_name = "step"
# event_column_name = "event"
# score_column_name = "score"
# use_scoring_intervals = None  # type:ignore


# # [1, 3, 5, 7, 10, 13, 15, 20, 25, 30] minute
# tolerances = {
#     "onset": [12, 36, 60, 90, 120, 150, 180, 240, 300, 360],
#     "wakeup": [12, 36, 60, 90, 120, 150, 180, 240, 300, 360],
# }

from dss_metrics import score

In [9]:
event_df = train_event_df[train_event_df["step"].notnull()].copy()

In [15]:
oof_df = pd.DataFrame()
for i in range(5):
    oof_df_fold = pd.read_parquet(os.path.join(output_dir, exp_name, f"oof_df_fold{i}.parquet"))
    df = postprocess_fn(oof_df_fold)
    sub_df = make_submission_df(df, threshold=0.01)
    event_fold_df = event_df[event_df["series_id"].isin(oof_df_fold["series_id"].unique())].copy()
    fold_score = score(event_fold_df, sub_df)
    print(f"fold{i} score:", fold_score)
    oof_df = pd.concat([oof_df, sub_df], axis=0)
oof_df = oof_df.reset_index(drop=True)

fold0 score: 0.6622583576166272
fold1 score: 0.666057093356913
fold2 score: 0.6405538667346251
fold3 score: 0.6514126108078131
fold4 score: 0.640819494291069


In [18]:
# df = postprocess_fn(oof_df)
# sub_df = make_submission_df(df, threshold=0.01)
event_all_df = event_df[event_df["series_id"].isin(oof_df["series_id"].unique())].copy()
all_score = score(event_all_df, oof_df)
print(f"all score:", all_score)

all score: 0.6483557411310146


In [ ]:
series_ids = set(event_df["series_id"].unique()) - set(oof_df["series_id"].unique())
print(len(series_ids))

In [ ]:
%%time
max_kernel_size = 41
tmp_score = 0
for average_size_ in range(101, 1001, 100):
    print("average_size", average_size_, "max_kernel_size", max_kernel_size, end=" ")
    df = postprocess_fn(oof_df, N=average_size_, maxpool_kernel_size=max_kernel_size)
    print("postprocessed", end=" ")
    sub_df = make_submission_df(df, threshold=0.01)
    print("submission", end=" ")
    score_ = score(event_df, sub_df)
    print(score_)
    if tmp_score > score_:
        break
    tmp_score = score_
    average_size = average_size_

threshold 0
average_size 101 max_kernel_size 41 postprocessed submission 0.6363470035624408
average_size 201 max_kernel_size 41 postprocessed submission 0.6499744712492275

In [ ]:
# tmp_score = 0
# for max_kernel_size in range(21, 201, 20):
#     print("average_size", average_size, "max_kernel_size", max_kernel_size)
#     df = postprocess_fn(oof_df, N=average_size, maxpool_kernel_size=max_kernel_size)
#     sub_df = make_submission_df(df, threshold=0.0)
#     score_ = score(event_df, sub_df)
#     print(score_)
#     if tmp_score > score_:
#         break
#     tmp_score = score_

In [ ]:
tmp_score = 0
for average_size_ in range(101, 1001, 100):
    print("average_size", average_size_)
    df = postprocess_fn(oof_df, N=average_size_, maxpool_kernel_size=3)
    sub_df = make_submission_df(df, threshold=0.01)
    score_ = score(event_df, sub_df)
    print(score_)
    if tmp_score > score_:
        break
    tmp_score = score_
    average_size = average_size_